# Literate operations

Inspired by [this post](http://howardism.org/Technical/Emacs/literate-devops.html), I'm trying to do 'literate operations' in JupyterLab.

For today, I'm trying to run a 10,000 users ramping up over 1 minute performance test of JupyterHub

## Deploy JupyterHub

First we deploy the Kubernetes Cluster + JupyterHub on GKE.

I edit 



In [4]:
cd ~/code/data8x-deploy

We create a fairly big cluster now

In [14]:
./bin/python deploy.py --config single-cluster.yaml --config secret.yaml --deployment single-10k gdm --create

The fingerprint of the deployment is 17rVUjI_PXFzSqy3KTHp8w==
Waiting for create [operation-1520716653981-567156ff6ed48-6fd417c2-46e2691b]...
...........................................................................................................................................................................done.
Create operation operation-1520716653981-567156ff6ed48-6fd417c2-46e2691b completed successfully.
NAME                       TYPE                       STATE      ERRORS  INTENT
single-10k-cluster-alpha   container.v1.cluster       COMPLETED  []
single-10k-db-instance     sqladmin.v1beta4.instance  COMPLETED  []
single-10k-db-proxyuser    sqladmin.v1beta4.user      COMPLETED  []
single-10k-nfs-sharder-db  sqladmin.v1beta4.database  COMPLETED  []


Let's see how many nodes we have!

In [15]:
kubectl get node 

Unable to connect to the server: dial tcp 35.225.120.245:443: i/o timeout


: 1

Hmm, not sure why this is happening. Let's look...

In [16]:
gcloud container clusters list

NAME                      ZONE           MASTER_VERSION  MASTER_IP       MACHINE_TYPE   NODE_VERSION   NUM_NODES  STATUS
single-10k-cluster-alpha  us-central1-a  1.9.2-gke.1     35.202.186.47   n1-standard-4  1.9.2-gke.1    100        RUNNING
test-hub-cluster-alpha    us-central1-a  1.8.7-gke.0     35.193.217.192  n1-highmem-2   1.8.6-gke.0 *  2          RUNNING

* - There is an upgrade available for your cluster(s).

To upgrade nodes to the latest available version, run
  $ gcloud container clusters upgrade NAME


In [20]:
# This is in status 'RUNNING, so we should be fine.
gcloud container operations list | grep single-10k
# Operation says done, so maybe I just need to wait a minute or so for the cluster to come up, since it has a 100 nodes?

operation-1520716667203-162181d3  CREATE_CLUSTER      us-central1-a  single-10k-cluster-alpha                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 DONE    2018-03-10T21:17:47.203183825Z  2018-03-10T21:19:58.369911909Z


In [23]:
kubectl get node

The connection to the server 35.225.120.245 was refused - did you specify the right host or port?


: 1

I just noticed that the IP kubectl is trying to connect to is different from the IP that is output from gcloud container clusters list! Doh! I need to explicitly get credentials for this cluster before I can use it! I normally don't need to, since I run the `init_support` command instead...

In [24]:
gcloud container clusters get-credentials single-10k-cluster-alpha

Fetching cluster endpoint and auth data.
kubeconfig entry generated for single-10k-cluster-alpha.


In [25]:
kubectl get node | wc -l

101


That's much better!

Now let's set up the 'support' installation. This contains prometheus and stuff, which we don't need right now. But let's set it up anyway...

In [26]:
./bin/python deploy.py --config single-cluster.yaml --config secret.yaml --deployment single-10k init_support

Fetching cluster endpoint and auth data.
kubeconfig entry generated for single-10k-cluster-alpha.
clusterrolebinding "cluster-admin-binding" created
serviceaccount "tiller" created
clusterrolebinding "tiller" created
$HELM_HOME has been configured at /home/yuvipanda/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.
Happy Helming!
Waiting for rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out
Hang tight while we grab the latest from your chart repositories...
...Unable to get an update from the "local" chart repository (http://127.0.0.1:8879/charts):
	Get http://127.0.0.1:8879/charts/index.yaml: dial tcp 127.0.0.1:8879: getsockopt: connection refused
...Unable to get an update from the "wat" chart repository (http://localhost:8000/gh-pages):
	Get http://localhost:8000/gh-pages/index.yaml: dial tcp 127.0.0.1:8000: getsockopt: connection refused
...Successfully got an update from the

OK, now we actually deploy!

In [27]:
cat single-cluster.yaml

region: us-central1

clusters:
  cluster-alpha:
    machineType: n1-standard-4
    bootDiskSizeGb: 100
    initialNodeCount: 100
    zone: us-central1-a
    hubs:
      hub-a: {}
      hub-b: {}
      hub-c: {}
      hub-d: {}
      hub-e: {}
      hub-f: {}
      hub-g: {}
      hub-h: {}
      hub-i: {}
      hub-j: {}
      hub-k: {}
      hub-l: {}
      hub-m: {}
      hub-n: {}
      hub-o: {}
      hub-p: {}
      hub-q: {}
      hub-r: {}
      hub-s: {}
      hub-t: {}
      hub-u: {}
      hub-v: {}
      hub-w: {}
      hub-x: {}
      hub-y: {}
      hub-z: {}
      
fileservers: {}
#  fileserver-01:
#    machineType: n1-highcpu-8
#    dataDiskSizeGb: 1000
#    zone: us-central1-a

project: data8x-scratch
admins:
  - YuviPanda@gmail.com


In [31]:
./bin/python deploy.py --config single-cluster.yaml --config secret.yaml --deployment single-10k deploy

"jupyterhub" has been added to your repositories
Fetching cluster endpoint and auth data.
kubeconfig entry generated for single-10k-cluster-alpha.
Hang tight while we grab the latest from your chart repositories...
...Unable to get an update from the "wat" chart repository (http://localhost:8000/gh-pages):
	Get http://localhost:8000/gh-pages/index.yaml: dial tcp 127.0.0.1:8000: getsockopt: connection refused
...Unable to get an update from the "local" chart repository (http://127.0.0.1:8879/charts):
	Get http://127.0.0.1:8879/charts/index.yaml: dial tcp 127.0.0.1:8879: getsockopt: connection refused
...Successfully got an update from the "jupyterhub" chart repository
...Successfully got an update from the "stable" chart repository
Update Complete. ⎈Happy Helming!⎈
Saving 1 charts
Deleting outdated charts
Release "hub-e" has been upgraded. Happy Helming!
Release "hub-d" has been upgraded. Happy Helming!
LAST DEPLOYED: Sat Mar 10 13:41:55 2018
NAMESPACE: hub-e
STATUS: DEPLOYED

RESOURCES

: 1

In [34]:
# A bunch of that failed, I think because I was trying to run upto 16 helm processes at once. I'm trying with 8 now.
./bin/python deploy.py --config single-cluster.yaml --config secret.yaml --deployment single-10k deploy

"jupyterhub" has been added to your repositories
Fetching cluster endpoint and auth data.
kubeconfig entry generated for single-10k-cluster-alpha.
Hang tight while we grab the latest from your chart repositories...
...Unable to get an update from the "local" chart repository (http://127.0.0.1:8879/charts):
	Get http://127.0.0.1:8879/charts/index.yaml: dial tcp 127.0.0.1:8879: getsockopt: connection refused
...Unable to get an update from the "wat" chart repository (http://localhost:8000/gh-pages):
	Get http://localhost:8000/gh-pages/index.yaml: dial tcp 127.0.0.1:8000: getsockopt: connection refused
...Successfully got an update from the "jupyterhub" chart repository
...Successfully got an update from the "stable" chart repository
Update Complete. ⎈Happy Helming!⎈
Saving 1 charts
Deleting outdated charts
Release "hub-e" has been upgraded. Happy Helming!
LAST DEPLOYED: Sat Mar 10 13:53:24 2018
NAMESPACE: hub-e
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/ClusterRoleBinding
NAME         AGE


In [35]:
# This also had some failures, which reminded me that I was trying to make all services be internal load balancers earlier! This is likely the cause...
# So I edit templates/values.yaml to remove that.
# Let's delete all services in hub namespaces first so the internal loadbalancers get released. We don't have enough quota for them all
kubectl get ns | awk '{print $1}' | grep hub | xargs -L1  -P0 kubectl delete svc proxy-public --now --namespace

service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted
service "proxy-public" deleted


In [36]:
./bin/python deploy.py --config single-cluster.yaml --config secret.yaml --deployment single-10k deploy

"jupyterhub" has been added to your repositories
Fetching cluster endpoint and auth data.
kubeconfig entry generated for single-10k-cluster-alpha.
Hang tight while we grab the latest from your chart repositories...
...Unable to get an update from the "local" chart repository (http://127.0.0.1:8879/charts):
	Get http://127.0.0.1:8879/charts/index.yaml: dial tcp 127.0.0.1:8879: getsockopt: connection refused
...Unable to get an update from the "wat" chart repository (http://localhost:8000/gh-pages):
	Get http://localhost:8000/gh-pages/index.yaml: dial tcp 127.0.0.1:8000: getsockopt: connection refused
...Successfully got an update from the "jupyterhub" chart repository
...Successfully got an update from the "stable" chart repository
Update Complete. ⎈Happy Helming!⎈
Saving 1 charts
Deleting outdated charts
Release "hub-b" has been upgraded. Happy Helming!
Release "hub-d" has been upgraded. Happy Helming!
LAST DEPLOYED: Sat Mar 10 14:03:36 2018
NAMESPACE: hub-b
STATUS: DEPLOYED

RESOURCES

: 1

Edge installation failed because it needs a public IP + forwarding rules, and we have a tiny quota for that. I'm waiting on Google to give me more quota. In the meantime, I'm going to the google cloud console and deleting some unused ones...

In [40]:
# That worked, and we now have a public IP!
kubectl --namespace=edge describe svc edge-proxy

Name:                     edge-proxy
Namespace:                edge
Labels:                   app=edge
                          chart=edge-0.1.0
                          component=proxy
                          heritage=Tiller
                          release=edge
Annotations:              <none>
Selector:                 app=edge,chart=edge-0.1.0,component=proxy,heritage=Tiller,release=edge
Type:                     LoadBalancer
IP:                       10.3.241.2
LoadBalancer Ingress:     35.224.63.98
Port:                     http  80/TCP
TargetPort:               80/TCP
NodePort:                 http  30928/TCP
Endpoints:                10.0.0.9:80,10.0.1.9:80,10.0.100.6:80 + 23 more...
Session Affinity:         None
External Traffic Policy:  Cluster
Events:
  Type    Reason                Age   From                Message
  ----    ------                ----  ----                -------
  Normal  EnsuringLoadBalancer  2m    service-controller  Ensuring load balancer
  Normal 

In [41]:
# Check if all pods are running
kubectl get --all-namespaces pod | grep -v Running

NAMESPACE         NAME                                                             READY     STATUS    RESTARTS   AGE


In [42]:
# Check if we can route to the edge proxy!
curl -I http://35.224.63.98

HTTP/1.1 405 Method Not Allowed
Server: openresty/1.13.6.1
Date: Sat, 10 Mar 2018 22:15:33 GMT
Content-Type: text/html
Content-Length: 3425
Connection: keep-alive
x-jupyterhub-version: 0.8.1
content-security-policy: frame-ancestors 'self'; report-uri /hub/security/csp-report
Strict-Transport-Security: max-age=15724800; includeSubDomains;



In [43]:
# We can! This is good :)

In [47]:
# Now we start load testing! I'm going to edit the hubtraf helm chart's file out of band.
# I want to run 1000 completions over 5 minutes.
# I realize now that I need about 125 nodes, rather than 100. So I'm going to do that first, by tweaking my config
./bin/python deploy.py --config single-cluster.yaml --config secret.yaml --deployment single-10k gdm

The fingerprint of the deployment is TTorIfdN2ePtTbvbUG5dAA==
Waiting for update [operation-1520720513130-5671655fcde11-2a61af46-5b9486de]...
...................................................................................................................................done.
Update operation operation-1520720513130-5671655fcde11-2a61af46-5b9486de completed successfully.
NAME                       TYPE                       STATE      ERRORS  INTENT
single-10k-cluster-alpha   container.v1.cluster       COMPLETED  []
single-10k-db-instance     sqladmin.v1beta4.instance  COMPLETED  []
single-10k-db-proxyuser    sqladmin.v1beta4.user      COMPLETED  []
single-10k-nfs-sharder-db  sqladmin.v1beta4.database  COMPLETED  []


In [52]:
gcloud container clusters resize single-10k-cluster-alpha --size=125

Pool [default] for [single-10k-cluster-alpha] will be resized to 125.

Do you want to continue (Y/n)?  

Command killed by keyboard interrupt




In [53]:
# Meh, forgot bash-kernel doesn't do prompts! Let's disable prompts
export CLOUDSDK_CORE_DISABLE_PROMPTS=1

In [55]:
gcloud container clusters resize single-10k-cluster-alpha --size=125

Resizing single-10k-cluster-alpha...done.
Updated [https://container.googleapis.com/v1/projects/data8x-scratch/zones/us-central1-a/clusters/single-10k-cluster-alpha].


In [58]:
# Let's see how this is going...
kubectl get node | wc -l

126


In [60]:
# Done! Now let's do hubtraf!
cd ~/code/hubtraf
ls

10s                 config.yaml  hubtraf           logs          README.md
10s.svg             Dockerfile   hubtraf.egg-info  Pipfile       setup.py
5000-users-60s.log  helm-chart   LICENSE           Pipfile.lock  values.yaml


In [64]:
# 1000 completions of 100 simulated users each, started uniformly over 10min
# Running between 5-10 mins for each user
cat config.yaml

completions: 1000
image:
  repository: yuvipanda/hubtraf
  tag: v1
resources: {}

hub:
  url: "http://edge-proxy.edge"

users:
  count: 100
  startTime:
    max: 600
  runTime:
    min: 300
    max: 600


In [65]:
# Kick off the test!
helm upgrade --install hubtraf helm-chart --namespace=hubtraf -f config.yaml

Release "hubtraf" does not exist. Installing it now.
NAME:   hubtraf
LAST DEPLOYED: Sat Mar 10 14:41:20 2018
NAMESPACE: hubtraf
STATUS: DEPLOYED

RESOURCES:
==> v1beta2/Deployment
NAME       DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
collector  1        1        1           0          0s

==> v1/Job
NAME             DESIRED  SUCCESSFUL  AGE
hubtraf-hubtraf  1000     0           0s

==> v1/Pod(related)
NAME                   READY  STATUS    RESTARTS  AGE
hubtraf-hubtraf-2cgw8  0/2    Init:0/1  0         0s
hubtraf-hubtraf-4h689  0/2    Init:0/1  0         0s
hubtraf-hubtraf-4hn5f  0/2    Init:0/1  0         0s
hubtraf-hubtraf-5jq5t  0/2    Init:0/1  0         0s
hubtraf-hubtraf-5vj99  0/2    Init:0/1  0         0s
hubtraf-hubtraf-6zr7h  0/2    Init:0/1  0         0s
hubtraf-hubtraf-755pf  0/2    Init:0/1  0         0s
hubtraf-hubtraf-79265  0/2    Init:0/1  0         0s
hubtraf-hubtraf-7w9c4  0/2    Init:0/1  0         0s
hubtraf-hubtraf-9v9pf  0/2    Init:0/1  0         0s
hubtraf-h

In [66]:
 kubectl --namespace=hubtraf get pod

NAME                         READY     STATUS            RESTARTS   AGE
collector-858979685d-64gcn   1/1       Running           0          12s
hubtraf-hubtraf-22fcs        0/2       PodInitializing   0          10s
hubtraf-hubtraf-22mvk        0/2       Init:0/1          0          3s
hubtraf-hubtraf-26xl9        0/2       PodInitializing   0          12s
hubtraf-hubtraf-2784n        0/2       Init:0/1          0          2s
hubtraf-hubtraf-28v2t        0/2       PodInitializing   0          11s
hubtraf-hubtraf-2c8sc        0/2       PodInitializing   0          7s
hubtraf-hubtraf-2cgw8        0/2       PodInitializing   0          12s
hubtraf-hubtraf-2fz77        0/2       PodInitializing   0          11s
hubtraf-hubtraf-2xq77        0/2       PodInitializing   0          9s
hubtraf-hubtraf-44lpf        0/2       Init:0/1          0          6s
hubtraf-hubtraf-47n7w        0/2       PodInitializing   0          5s
hubtraf-hubtraf-4cl64        0/2       Init:0/1          0          10

In [69]:
# Wait for all pods to be created! 
kubectl --namespace=hubtraf get pod | wc -l

1002


In [75]:
# Take a look at the collected logs
COLLECTOR=$(kubectl --namespace=hubtraf get pod -l component=collector -o name)
# Since bash_kernel doesn't erally do autocomplete, let's use a variable to 
# capture the name of the collector

In [76]:
# Be *very* careful to not see full logs here, otherwise browser *will* crash
kubectl --namespace=hubtraf logs ${COLLECTOR} | tail 

[1394] tail.0: [1520722329.382116662, {"log"=>"{"username": "hubtraf-hubtraf-tw26s-7", "hub": "hub-n", "action": "start-server", "phase": "failed", "duration": 368.99935783100045, "event": "Server: Failed", "timestamp": "2018-03-10T22:52:09.381855Z"}"}]
[1395] tail.0: [1520722325.249974711, {"log"=>"{"username": "hubtraf-hubtraf-796bh-96", "action": "login", "phase": "start", "event": "Login: Starting", "timestamp": "2018-03-10T22:52:05.224772Z"}"}]
[1396] tail.0: [1520722326.196382923, {"log"=>"{"username": "hubtraf-hubtraf-796bh-96", "hub": "hub-s", "action": "start-server", "phase": "start", "attempt": 2, "event": "Server: Starting 1", "timestamp": "2018-03-10T22:52:06.195887Z"}"}]
[1397] tail.0: [1520722326.601510970, {"log"=>"{"username": "hubtraf-hubtraf-796bh-62", "hub": "hub-s", "action": "start-server", "phase": "failed", "duration": 339.10869916499996, "event": "Server: Failed", "timestamp": "2018-03-10T22:52:06.444810Z"}"}]
[1398] tail.0: [1520722326.940905354, {"log"=>"{"us

In [77]:
# I see failed events! Let's look at how many failed start-server events there are
kubectl --namespace=hubtraf logs ${COLLECTOR} | grep start-server | grep -c failed

589


In [79]:
# Is it growing or stable?
kubectl --namespace=hubtraf logs ${COLLECTOR} | grep start-server | grep -c failed

948


In [81]:
# Whoops, it's growing intensely :) Let's see what pods are in non-running state
kubectl get --all-namespaces pod | grep -v Running

Stream error http2.StreamError{StreamID:0x1, Code:0x2, Cause:error(nil)} when reading response body, may be caused by closed connection. Please retry.


: 1

In [82]:
# whoops, our master is overwhelmed. This adds credence to mytheory that a single k8s cluster is not
# going to work for us. But we should also remember, we're doing an intense load test - 10k users in 5mins
# is not what we actually expect to happen!
# Let's see if we had triggered autoscaling, to rule out resource exhaustion as reason for failures
kubectl get node | wc -l

152


In [83]:
# We did! We started with 125 nodes, but are now at 151! This clearly means my math on how many nodes we
# need was wildly out of whack.
# Each pod needed 128M, and there's a 100 pod limit per node. Assuming we can use 90 pods per node,
# 10,000 pods need:
echo "10000 / 90" | bc

111


In [84]:
# So we needed minimum of 111 nodes, but clearly that wasn't enough.
# I added another 15 nodes to account for the 1000 stress test pods, but that wasn't enough?
# Somewhere my math is wrong, need to go and form a proper set up for this
# Let's see how many nodes we have now
kubectl get node | wc -l

167


In [85]:
# Maybe there's a bug in our stress tester code that's creating far more servers than needed?
kubectl get pod --all-namespaces | grep -c jupyter

14116


In [86]:
# Yup, ugh, that seems to be the problem! This needs more debugging...
# Let's tear down the cluster while we debug, so we don't end up spending too much time.

# Get rid of the stress test pods
kubectl --namespace=hubtraf delete job --all

job "hubtraf-hubtraf" deleted


In [89]:
# The teardown script removes everything cleanly!
cd ~/code/data8x-deploy
./bin/python deploy.py --config single-cluster.yaml --config secret.yaml --deployment single-10k teardown

Fetching cluster endpoint and auth data.
kubeconfig entry generated for single-10k-cluster-alpha.
deployment "cluster-support-prometheus-kube-state-metrics" deleted
deployment "cluster-support-prometheus-server" deleted
persistentvolumeclaim "cluster-support-prometheus-server" deleted
deployment "hub" deleted
persistentvolumeclaim "hub-db-dir" deleted
deployment "hub" deleted
persistentvolumeclaim "hub-db-dir" deleted
deployment "hub" deleted
persistentvolumeclaim "hub-db-dir" deleted
deployment "hub" deleted
persistentvolumeclaim "hub-db-dir" deleted
deployment "hub" deleted
persistentvolumeclaim "hub-db-dir" deleted
deployment "hub" deleted
persistentvolumeclaim "hub-db-dir" deleted
deployment "hub" deleted
persistentvolumeclaim "hub-db-dir" deleted
deployment "hub" deleted
persistentvolumeclaim "hub-db-dir" deleted
deployment "hub" deleted
persistentvolumeclaim "hub-db-dir" deleted
deployment "hub" deleted
persistentvolumeclaim "hub-db-dir" deleted
deployment "hub" deleted
persisten

# What did we learn in this? It's not entirely clear!

1. A single, non-regional kubernetes cluster is too much of a risk to use for production. We must have multiple ones, preferably regional clusters
2. There's a bug in our stress tester code that spawns far more pods than needed. Investigate what this is. Possibly driven by 504 errors from the edge proxies being overloaded?
3. An earlier experiment I did (using Internal Load Balancers, one per hub) is not going to work, since that has too high a quota overhead (for negligble advantages)

Mostly, this was an exercise in using JupyterLab to have this kinda notebook. There are lots of issues left to be worked out, but the format in which it forces you to think feels extremely helpful! I'm keeping a running list of issues that'll make this experience easier in [this HackMD](https://hackmd.io/1HzvdWibQ6K2KmIWNBBvZA?edit)
